<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling 
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (You will be implementing the decoder, not the encoder, as we are not doing sequence-to-sequence translation.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [1]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2021-10-16 23:58:11--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 52.54.75.23, 52.44.149.188, 54.86.243.162, ...
Connecting to piazza.com (piazza.com)|52.54.75.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2021-10-16 23:58:11--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 13.249.93.52, 13.249.93.39, 13.249.93.35, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|13.249.93.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  --.-KB/s    in 0

In [2]:
import gc

In [3]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

ide on and tell 
Fatty Bolger that you are coming. We'll see about supper and things.' 

'We had our supper early with Farmer Maggot,' said Frodo; 'but we could 
do with another.' 

'You shall have it!


In [4]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor(random_chunk()))


tensor([94, 40, 10, 16, 21, 14, 28, 73, 94, 12, 10, 22, 14, 94, 30, 23, 21, 24,
        24, 20, 14, 13, 74, 15, 24, 27, 94, 29, 24, 94, 96, 50, 27, 29, 17, 10,
        23, 12, 78, 94, 10, 23, 13, 94, 17, 14, 94, 15, 24, 30, 23, 13, 94, 22,
        14, 94, 28, 29, 10, 23, 13, 18, 23, 16, 94, 24, 23, 94, 29, 17, 14, 94,
        25, 18, 23, 23, 10, 12, 21, 14, 75, 94, 55, 17, 14, 23, 94, 44, 94, 28,
        25, 24, 20, 14, 94, 29, 24, 94, 17, 18, 22, 94, 10, 23, 13, 94, 96, 17,
        14, 94, 11, 24, 27, 14, 94, 22, 14, 94, 10, 32, 10, 34, 73, 94, 11, 14,
        15, 24, 27, 14, 94, 54, 10, 27, 30, 22, 10, 23, 94, 32, 10, 28, 94, 10,
        32, 10, 27, 14, 75, 94, 44, 94, 32, 10, 28, 94, 15, 10, 27, 94, 15, 27,
        24, 22, 94, 44, 28, 14, 23, 16, 10, 27, 13, 73, 94, 14, 27, 14, 94, 29,
        17, 14, 94, 96, 32, 24, 21, 31, 14, 28, 94, 10, 23, 13, 94, 24, 27, 14,
        28, 94, 18])


---

## Part 4: Creating your own GRU cell 

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please do not look at the documentation's code for the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**
* Create a custom GRU cell

**DONE:**



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    
  
  def forward(self, inputs, hidden):
    # Each layer does the following:
    # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
    # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
    # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
    # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
    # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)
    
    return outputs, hiddens
  


---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**
* Create an RNN class that extends from nn.Module.

**DONE:**



In [6]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    
    
    # more stuff here...
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.sequence = nn.Sequential()
    self.gru = nn.GRU(hidden_size, hidden_size, num_layers = n_layers)
    #for i in range(n_layers-1):
      #self.sequence.add_module(str(i+1) + "th Gru", nn.GRU(input_size, output_size))


  def forward(self, input_char, hidden):
    #print("Input Char in RNN forward: ", input_char)
    output = self.embedding(input_char).view(1, 1, -1)
    #print(output.size())
    #output = output.unsqueeze(0).unsqueeze(0)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden) #change this to sequence?
    output.squeeze(0).squeeze(0)

    output = self.softmax(self.out(output[0]))
    return output, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [7]:
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes. 

**TODO:**
* Fill in the pieces.

**DONE:**




In [8]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target):
  ## initialize hidden layers, set up gradient and loss 
    # your code here
  ## /
  inp = inp.cuda()
  target = target.cuda()

  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  hidden = hidden.cuda()
  loss = 0


  
  #output, hidden = decoder(inp[0],hidden)
  #loss += criterion(output, target[0].unsqueeze(0))

  for i in range(len(inp)):
    output, hidden = decoder(inp[i],hidden)
    loss += criterion(output,target[i].unsqueeze(0).long())
  all_losses.append(loss)

  loss.backward()
  decoder_optimizer.step()

  gc.collect()
  return loss / len(inp)
  



    
  # more stuff here...

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**
* Fill out the evaluate function to generate text frome a primed string

**DONE:**



In [13]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    # As temperature approaches 0, this sampling function becomes argmax (no randomness)
    # As temperature approaches infinity, this sampling function becomes a purely random choice
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  hidden = decoder.init_hidden()
  hidden = hidden.cuda()
  prime = char_tensor(prime_str)

  ## initialize hidden state, initialize other useful variables
    # your code here
  ## /
  output_str = ""
  for c in prime:
    c = c.cuda()
    output, hidden = decoder(c, hidden)
    output_str += all_characters[c]
  char = sample_outputs(output, temperature)
  output_str += all_characters[char]
  for _ in range(predict_len):
    output, hidden = decoder(char, hidden)
    char = sample_outputs(output, temperature)
    output_str += all_characters[char]

    gc.collect()
  return output_str

  

  

---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:** 
* Create some cool output

**DONE:**




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:
 
 Gandalf was decrond. 
'All have lord you. Forward the road at least walk this is stuff, and 
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long 
row hrough. In  

 lo:
 
 lost death it. 
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate. 
'Yes, as you there were remembaused to seen their pass, when? What 
said here, such seven an the sear 

 lo:
 
 low, and frod to keepn 
Came of their most. But here priced doubtless to an Sam up is 
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of 
the like 

 I:
 
 I had been the 
in his eyes with the perushed to lest, if then only the ring and the legended 
of the less of the long they which as the 
enders of Orcovered and smood, and the p 

 I:
 
 I they were not the lord of the hoomes. 
Home already well from the Elves. And he sat strength, and we 
housed out of the good of the days to the mountains from his perith. 

'Yess! Where though as if  

 Th:
 
 There yarden 
you would guard the hoor might. Far and then may was 
croties, too began to see the drumbred many line 
and was then hoard walk and they heart, and the chair of the 
Ents of way, might was 

 G:
 
 Gandalf 
been lat of less the round of the stump; both and seemed to the trees and perished they 
lay are speered the less; and the wind the steep and have to she 
precious. There was in the oonly went 

 wh:
 
 which went out of the door. 
Hull the King and of the The days of his brodo 
stumbler of the windard was a thing there, then it been shining langing 
to him poor land. They hands; though they seemed ou 

 ra:
 
 rather,' have all the least deather 
down of the truven beginning to the house of sunk. 
'Nark shorts of the Eyes of the Gate your great nothing as Eret. 
'I wander trust horn, and there were not, it  

 I:
 
 I can have no mind 
together! Where don't may had one may little blung 
terrible to tales. And turn and Gandalf shall be not to as only the Cattring 
not stopped great the out them forms. On they she lo 

---


In [10]:
import time
n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3 #what do I do with this?
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder.cuda()
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [11]:
n_epochs = 2000
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[83.28537631034851 (200 10%) 2.6583]
 yesh otheud.. same soan the 
an 
sa, bFe, ud wome nice 'saltas wound 
kas an onh anr opow', 

ase ar 

[170.4433662891388 (400 20%) 2.5282]
er of the is sele to the and bose the buble fare it for goble. But a lat the if there hich the his of 

[257.2510793209076 (600 30%) 1.8895]
 mum the stong and looge his and I mam with the sfore 
leins whis hipper and 
cuor at neach the one l 

[344.39677357673645 (800 40%) 1.8488]
ere well sunded of the with candting ssondenf Frodo, and come the would the came. 'A he bet of abgo h 

[432.4792342185974 (1000 50%) 1.7894]
at of the tounds in the cought. The hook his had sward the sees and the 
more of Menn, and tle eppons 

[520.5570111274719 (1200 60%) 1.9227]
ere was suraming some like of the 
rack 
again. He we tord us 
in blone and have 
and into not mate 
 

[606.7667422294617 (1400 70%) 1.6364]
e hime ance as hjil om 
that haln not trung; I the light of the 

diaming whonelf in the holbo rough  

[692.7205

In [14]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 he
 her. Strong and was rived a long 
shadown tide a beal wonder his spare great and stross. He said his 
porner it. The 
hills and the passobe. 

'Now we halflys had befire them, and strided the horsind bir 

 he
 her. 

And cander longed to 
hobbits bear.' 

Heard if was heard the dranger. 'I has 
every?' he say about it the gate of still 
has stranged viguntel, for the he seem it were slitring to were to day 
to 

 G
 Gandalf 
in strange is see way the ores of with the will not 
his a fierd. As they that for thefe has stered Midden, not almost to a and vil of strange wild, and decome it. Them 
not in ruin for a darre 

 G
 Gimli and bean, if the stand 
Northessed. 
He tall from meant from the doind, and cave 
















hows down 
to the hands of the will that strong was living 
have of 
their 
least were not more to 

 ca
 causen the 
horder Boroom than he masped the ance of for of the was 
hidden hidden rulf night-so name the said the a how 


and beep day drind. 

'You sand

---

## Part 6: Generate output on a different dataset

---

**TODO:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle 

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)

**DONE:**

